# BB84 Quantum Key Distribution (QKD) Protocol

Quantum Key Distribution (QKD) is a method used to securely share cryptographic keys between two parties, typically called Alice (the sender) and Bob (the receiver). The security of QKD is guaranteed by the fundamental principles of quantum mechanics, particularly the no-cloning theorem and the uncertainty principle. The BB84 protocol is one of the most well-known and simplest QKD protocols, proposed by Charles Bennett and Gilles Brassard in 1984.

### Steps of the BB84 Protocol

1. Qubit Preparation: Alice encodes random bits into qubits using two possible bases (rectilinear and diagonal).
2. Quantum Communication: Alice sends the qubits to Bob over a quantum channel.
3. Measurement: Bob randomly measures the qubits using one of the two bases.
4. Basis Announcement: Alice and Bob publicly announce their bases and discard mismatched bits.
5. Error Checking: They compare a subset of the remaining bits to check for eavesdropping (error detection).
6. Key Generation: If the error rate is low, the remaining bits form a secure cryptographic key.

In [27]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import random
import hashlib

# Number of qubits (or bits)
num_bits = 10
error_detection_sample_size = 2

# Step 1: Alice generates random bits and random bases
alice_bits = np.random.randint(2, size=num_bits)  # Alice's random bits (0 or 1)
alice_bases = np.random.randint(2, size=num_bits)  # Alice's random bases (0: rectilinear, 1: diagonal)

print("Alice's bits: ", alice_bits)
print("Alice's bases: ", alice_bases)


Alice's bits:  [0 1 0 1 1 1 1 1 1 1]
Alice's bases:  [1 1 1 1 1 1 1 1 0 0]


In [28]:
def prepare_qubit(bit, basis):
    """Prepare qubit based on the bit and basis."""
    qc = QuantumCircuit(1, 1)
    if bit == 1:
        qc.x(0)  # Apply X gate if bit is 1 (i.e., prepare |1>)
    if basis == 1:
        qc.h(0)  # Apply Hadamard gate for diagonal basis
    return qc


In [29]:
# Step 2: Bob generates random bases
bob_bases = np.random.randint(2, size=num_bits)  # Bob's random bases (0: rectilinear, 1: diagonal)
print("Bob's bases: ", bob_bases)

# Step 3: Bob measures the qubits
def measure_qubit(qc, basis):
    """Measure qubit in the given basis."""
    if basis == 1:
        qc.h(0)  # Apply Hadamard gate for diagonal basis
    qc.measure(0, 0)  # Measure the qubit
    return qc


# Initialize simulator
simulator = AerSimulator()

bob_results = []  # Bob's measurement results

for i in range(num_bits):
    # Step 1: Alice prepares the qubit
    qc = prepare_qubit(alice_bits[i], alice_bases[i])
    
    # Step 2: Bob measures the qubit
    qc = measure_qubit(qc, bob_bases[i])
    
    compiled_circuit = transpile(qc, simulator)
    # Step 3: Execute the circuit
    result = simulator.run(compiled_circuit, shots=1).result()
    counts = result.get_counts()
    
    # Get Bob's measurement result
    measured_bit = int(list(counts.keys())[0])  # Extracting the measured bit (0 or 1)
    bob_results.append(measured_bit)

print("Bob's results: ", bob_results)


Bob's bases:  [1 1 1 0 1 1 0 0 0 0]
Bob's results:  [0, 1, 0, 1, 1, 1, 0, 1, 1, 1]


In [30]:
# Step 4: Alice and Bob compare their bases
shared_key = []
matching_indices = []

for i in range(num_bits):
    if alice_bases[i] == bob_bases[i]:  # Only keep bits where the bases match
        shared_key.append(bob_results[i])
        matching_indices.append(i)

print("Shared key (before error detection): ", shared_key)


Shared key (before error detection):  [0, 1, 0, 1, 1, 1, 1]


# Error detection

Error detection ensures that Alice and Bob can verify the integrity of their shared key and detect any potential eavesdropping attempts by Eve. This is typically done by comparing a portion of the shared key over the classical channel. If too many discrepancies are found, they discard the key.

In [31]:
# Alice and Bob reveal and compare a portion of their shared key to check for errors
error_check_indices = random.sample(matching_indices, min(error_detection_sample_size, len(matching_indices)))

error_detected = False
for index in error_check_indices:
    if alice_bits[index] != bob_results[index]:
        error_detected = True
        print(f"Error detected at index {index}. Discarding key.")
        break

# If errors are detected, discard the key
if error_detected:
    shared_key = []
else:
    print("No errors detected. Proceeding to privacy amplification.")

No errors detected. Proceeding to privacy amplification.


# Privacy amplification

Privacy amplification is used to reduce the information that an eavesdropper might have learned about the key. This is done by shrinking the shared key using a cryptographic hash function, effectively minimizing any leaked information and producing a shorter, secure key.

In [32]:
# Using a simple cryptographic hash function (SHA-256) to generate a new, shorter key
def privacy_amplification(key):
    key_str = ''.join(map(str, key))  # Convert key to a string
    hash_object = hashlib.sha256(key_str.encode())  # Hash the key string
    hashed_key = hash_object.hexdigest()  # Get the hash value in hexadecimal
    # Convert the hex digest to a binary string and use the first part as the new key
    new_key = bin(int(hashed_key, 16))[2:]  # Convert hex to binary
    return new_key[:len(key)]  # Truncate to the original key length (or shorter)

if shared_key:
    final_key = privacy_amplification(shared_key)
    print("Final key (after privacy amplification): ", final_key)
else:
    print("No final key generated due to error detection.")

Final key (after privacy amplification):  1010010
